<a href="https://colab.research.google.com/github/devamjariwala24/AWS-Certified-Cloud-Practitioner-Notes/blob/master/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings

In class, we discussed how we can reduce the dimensionality of word representations from their original vector space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classification or retrieval models.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [ ]:
!pip uninstall numpy -y
!pip uninstall scipy -y

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: scipy 1.13.1
Uninstalling scipy-1.13.1:
  Successfully uninstalled scipy-1.13.1


In [ ]:
!pip install numpy==1.26.4 --no-cache-dir
!pip install scipy==1.13.1 --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 80.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, which is not installed.
jaxlib 0.5.1 requires scipy>=1.11.1, which is not installed.
qdldl 0.1.7.post5 requires scipy>=0.13.2, which is not installed.
plotnine 0.14.5 requires scipy>=1.8.0, which is not installed.
shap 0.47.0 requires scipy, which is not installed.
treelite 4.4.1 requires scipy, which is not installed.
scs 3.2.7.post2 requires scipy, which is not installed.
mizani 0.13.1 requires scipy>=1.8.0, which is not installed.
umap-learn 0.5.7 requires scipy>=1.3.1, which is not installed.
sklearn-pandas 2.2.0 requires scipy>=1.5.1, which is not installed.
lightgbm 4.5.0 requires scipy, which is not install

In [ ]:
## If you already have gensim, you can comment this out.
!pip install gensim

In [ ]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the plays of William Shakespeare.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2025-03-28 00:43:45--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt.1’

shakespeare_plays.t 100%[===================>]   4.53M  9.07MB/s    in 0.5s    

2025-03-28 00:43:46 (9.07 MB/s) - ‘shakespeare_plays.txt.1’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.


In [ ]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy
tasks we discussed in class.

In [ ]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.7669879198074341),
 ('prince', 0.7248110175132751),
 ('duke', 0.7217379808425903),
 ('clarence', 0.7165194749832153),
 ('york', 0.708121657371521),
 ('warwick', 0.7045184373855591),
 ('cardinal', 0.6906446218490601),
 ('son', 0.6822353005409241),
 ('princess', 0.6643941402435303),
 ('northumberland', 0.6581059098243713)]

In other words, we want a vector close to `king` and `woman` but subtracting the dimensions that are important to `man`, i.e., `queen`. Other words are mostly noble titles important in Shakespeare's "history" plays.

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [ ]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

[('desdemona', 0.9690727591514587),
 ('emilia', 0.9156944751739502),
 ('iago', 0.909894585609436),
 ('ham', 0.8952130675315857),
 ('cassio', 0.8951020240783691),
 ('countess', 0.8919875025749207),
 ('cressida', 0.8913081884384155),
 ('pisanio', 0.8838783502578735),
 ('cleopatra', 0.8810870051383972),
 ('imogen', 0.8807059526443481)]

If you know the play, you might see some familiar names.

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [ ]:
model.wv.similarity('othello', 'desdemona')

0.9690728

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [ ]:
## TODO: Implement cosim
import numpy as np
def cosim(v1, v2):
  dot_product = np.dot(v1, v2)
  np_v1_norm = np.linalg.norm(v1)
  np_v2_norm = np.linalg.norm(v2)

  cos_product = dot_product / (np_v1_norm * np_v2_norm)
  return cos_product

## This should give a result similar to model.wv.similarity:
cosim_ans = cosim(model.wv['othello'], model.wv['desdemona'])
print(cosim_ans)

0.9690728


## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ru.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.zh.vec

--2025-03-28 00:43:59--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec.1’

30k.en.vec.1        100%[===================>]  64.54M  38.2MB/s    in 1.7s    

2025-03-28 00:44:01 (38.2 MB/s) - ‘30k.en.vec.1’ saved [67681172/67681172]

--2025-03-28 00:44:01--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67802327 (65M)
Saving to: ‘30k.fr.vec.1’

30k.fr.vec.1        100%[===================>]  64.66M  38.1MB/s    in 1.7s    

2025-03-28 00:44:03 (38.1 MB/s) - ‘30k.fr.vec.1’ saved [67802327/67802327]

--2025-03-28 00:44:03--  http://www.

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [ ]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1}

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this part of the assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [ ]:
def mostSimilar(vec, vecDict):
    mostSimilar = ''
    max_similarity = -1

    for word, word_vec in vecDict.items():
        similarity = cosim(vec, word_vec)

        if similarity > max_similarity:
            max_similarity = similarity
            mostSimilar = word

    return (mostSimilar, max_similarity)

In [ ]:
test_words = ['computer', 'germany', 'matrix', 'physics', 'yeast']
results = [mostSimilar(envec[e], frvec) for e in test_words]
for word, result in zip(test_words, results):
    print(f"Most similar to '{word}': {result}")

Most similar to 'computer': ('informatique', 0.5023827767603765)
Most similar to 'germany': ('allemagne', 0.593718413875964)
Most similar to 'matrix': ('matrice', 0.5088361302065517)
Most similar to 'physics': ('physique', 0.4555543434796394)
Most similar to 'yeast': ('fermentation', 0.3504105196166514)


In [ ]:
[mostSimilar(envec[e], esvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('computador', 0.5013697495254124),
 ('alemania', 0.6352798713596078),
 ('matriz', 0.4784864671614966),
 ('física', 0.4784845095690361),
 ('levadura', 0.5114917452709493)]

In [ ]:
dance_in_spanish = mostSimilar(envec['dance'], esvec)
print(dance_in_spanish)

('baile', 0.5620383041003675)


Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
links = [s.split() for s in open('links.tab')]

--2025-03-28 00:44:12--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab.1’

links.tab.1         100%[===================>]   1.34M  3.10MB/s    in 0.4s    

2025-03-28 00:44:13 (3.10 MB/s) - ‘links.tab.1’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [ ]:
links[302]

['academy', 'fr', 'académie']

We construct a test set for English-French from the first 1000 links between those languages.

In [ ]:
frtest = [x for x in links if x[1] == "fr"][0:1000]
frtest[0:10]

[['aalborg', 'fr', 'aalborg'],
 ['aarhus', 'fr', 'aarhus'],
 ['aba', 'fr', 'aba'],
 ['abad', 'fr', 'abad'],
 ['abandon', 'fr', 'abandon'],
 ['abbas', 'fr', 'abbas'],
 ['abbasid', 'fr', 'abbassides'],
 ['abbess', 'fr', 'abbesse'],
 ['abbey', 'fr', 'abbaye'],
 ['abbot', 'fr', 'abbé']]

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article in this test set `frtest` is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary.

Since many articles, e.g., about named entities, have the same title in English and French, use the identity function as a baseline and compute its accuracy. In other words, how often would you find the right French articles by simply echoing the English title as if it were French? In the ten example links above, `aalborg` and `aarhus` (two cities in Denmark) are the same in English and French. Remember to iterate only over the 1000 linked Wikipedia articles in the test set, not the entire embedding dictionary.

Here, I have followed this approach: We will iterate through the 1000 words of frtest and estest, if we get the word in the embedding, then only we wil consider that to be a valid link. This way, we will have real accuracy where accuracy is found by dividing with the number of valid links and not 1000, because if we divide it by 1000, then we will also be including those words which are skipped. By dividing with valid links, we include only those words for which we found in embedding.

In [ ]:
from tqdm import tqdm
accuracy = 0
baselineAccuracy = 0
valid_links = 0
exact_word_matches = 0
word_pairs_dict = {}

for link in tqdm(frtest, total=len(frtest), desc="Processing Links", unit="link"):
    english_word = link[0]
    french_word = link[2]
    word_pairs_dict[english_word] = french_word

    if english_word in envec:
        valid_links += 1
        most_similar_french_word, _ = mostSimilar(envec[english_word], frvec)
        if most_similar_french_word == french_word:
            accuracy += 1

        if english_word == french_word:
            baselineAccuracy += 1
            exact_word_matches += 1

if valid_links > 0:
    accuracy_percentage = (accuracy / valid_links) * 100
    print("\n")
    print(f"\nPrecision@1 for french is {accuracy_percentage:.2f}%")
    baseline_accuracy_percentage = (baselineAccuracy / valid_links) * 100
    print(f"Baseline Accuracy for french is {baseline_accuracy_percentage:.2f}%")
else:
    print("No valid links found with both embeddings.")

Processing Links: 100%|██████████| 1000/1000 [03:43<00:00,  4.47link/s]




Precision@1 for french is 57.50%
Baseline Accuracy for french is 66.10%


**TODO**: Compute the accuracy, i.e. precision@1, of the embeddings and of the identity function for the first 1000 links between English and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [ ]:
## TODO: Compute English-X Wikipedia retrieval accuracy.
estest = [x for x in links if x[1] == "es"][0:1000]

In [ ]:
accuracy = 0
baselineAccuracy = 0
valid_links = 0

for link in tqdm(estest, total=len(estest), desc="Processing Links", unit="link"):
    english_word = link[0]
    spanish_word = link[2]

    if english_word in envec:
        valid_links += 1
        most_similar_spanish_word, _ = mostSimilar(envec[english_word], esvec)
        if most_similar_spanish_word == spanish_word:
            accuracy += 1

        if english_word == spanish_word:
            baselineAccuracy += 1

if valid_links > 0:
    accuracy_percentage = (accuracy / valid_links) * 100
    print("\n")
    print(f"\nPrecision@1 for spanish is {accuracy_percentage:.2f}%")
    baseline_accuracy_percentage = (baselineAccuracy / valid_links) * 100
    print(f"Baseline Accuracy for spanish is {baseline_accuracy_percentage:.2f}%")
else:
    print("No valid links found with both embeddings.")

Processing Links: 100%|██████████| 1000/1000 [03:45<00:00,  4.44link/s]




Precision@1 for spanish is 53.60%
Baseline Accuracy for spanish is 51.80%


**TODO**: Find the 10 nearest neighbors of each English term to compute "recall at 10" and "mean reciprocal rank at 10".

In [ ]:
## TODO: Compute recall@10 and MRR@10 when retrieving 10 nearest neighbors in French and some other language.

In [ ]:
from collections import defaultdict
import heapq
import time
from tqdm import tqdm

def get_top_10_nearest_neighbors(word_vector, vec_dict):
    similarities = [(word, cosim(word_vector, vec)) for word, vec in vec_dict.items()]
    return heapq.nlargest(10, similarities, key=lambda x: x[1])

def process_links(test_links, envec, vec_dict, lang_type):
    valid_links = 0
    missing_embeddings = 0
    recall_at_10 = 0
    mrr_at_10 = 0
    top_10_neighbors_dict = defaultdict(list)

    start_time = time.time()
    for link in tqdm(test_links, total=len(test_links), desc=f"Processing {lang_type} Links", unit="link"):
        english_word = link[0]
        target_word = link[2]

        if english_word in envec:
            valid_links += 1
            top_10_neighbors_dict[english_word] = get_top_10_nearest_neighbors(envec[english_word], vec_dict)
        else:
            missing_embeddings += 1

    time_taken = time.time() - start_time

    for link in tqdm(test_links, total=len(test_links), desc=f"Calculating MRR & Recall for {lang_type}", unit="link"):
        english_word = link[0]
        target_word = link[2]

        if english_word in top_10_neighbors_dict:
            top_10_neighbors = top_10_neighbors_dict[english_word]
            top_10_words = [neighbor[0] for neighbor in top_10_neighbors]

            if target_word in top_10_words:
                recall_at_10 += 1

            for rank, (word, _) in enumerate(top_10_neighbors, start=1):
                if word == target_word:
                    mrr_at_10 += 1 / rank
                    break

    recall_at_10_percentage = recall_at_10 / valid_links if valid_links > 0 else 0
    mrr_at_10_average = mrr_at_10 / valid_links if valid_links > 0 else 0

    return recall_at_10_percentage, mrr_at_10_average, time_taken, missing_embeddings

recall_at_10_spanish, mrr_at_10_spanish, time_taken_spanish, missing_embeddings_spanish = process_links(estest, envec, esvec, "Spanish")
recall_at_10_french, mrr_at_10_french, time_taken_french, missing_embeddings_french = process_links(frtest, envec, frvec, "French")

print("\n")
print(f"\nRecall@10 for Spanish: {recall_at_10_spanish:.2f}")
print(f"MRR@10 for Spanish: {mrr_at_10_spanish:.2f}")
print(f"\nRecall@10 for French: {recall_at_10_french:.2f}")
print(f"MRR@10 for French: {mrr_at_10_french:.2f}")
print(f"\nTime taken to compute top 10 neighbors for Spanish: {time_taken_spanish:.4f} seconds")
print(f"Time taken to compute top 10 neighbors for French: {time_taken_french:.4f} seconds")

Calculating MRR & Recall for French: 100%|██████████| 1000/1000 [00:00<00:00, 164147.78link/s]




Recall@10 for Spanish: 0.61
MRR@10 for Spanish: 0.56

Recall@10 for French: 0.64
MRR@10 for French: 0.60

Time taken to compute top 10 neighbors for Spanish: 236.9498 seconds
Time taken to compute top 10 neighbors for French: 235.0180 seconds


## Speeding up Vector Search (required for CS6200, 20 points extra for IS4200)

The list of Wikipedia headwords is short enough that a linear scan through the non-English language embeddings takes some time but is feasible. In a production system, you could index the word embeddings using SimHash or some other locality sensitive hashing scheme, as we discussed for duplicate detection, to speed up this process.

A relatively easy way to get started with fast vector similarity search is to install Meta's `faiss` (Facebook AI Similarity Search) package and read [the tutorial](https://github.com/facebookresearch/faiss/wiki/Getting-started).

In [ ]:
# Outside of colab, you may need a different package manager.
# !apt install libomp-dev
!pip install --upgrade faiss-cpu
# Use this line instead if you have a GPU.
# !python -m pip install --upgrade faiss-gpu
import faiss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 19.8 MB/s eta 0:00:00


**TODO**: Create two vector indexes, for the FastText embeddings of French and for the other language you evaluated above. Use `faiss` to find the 10 nearest neighbors for the top 1000 Wikipedia headwords you evaluated for English-French and the English-X as above.

First, measure the _effectiveness_ of this approximate vector search approach. How does the R@10 and MRR@10 using `faiss` compare to the brute-force search you did above?

Second, measure the _efficiency_ of this approach. How long in seconds does finding nearest neighbors for 1000 headwords by brute force compare to using `faiss`? (For this exercise, don't worry about amortizing indexing costs.)

In [ ]:
import faiss
import numpy as np
from tqdm import tqdm
import time

def embeddings_to_numpy(embeddings_dict):
    words = list(embeddings_dict.keys())
    vectors = np.array([embeddings_dict[word] for word in words])
    return words, vectors

def create_faiss_index(embeddings_dict):
    words, vectors = embeddings_to_numpy(embeddings_dict)
    index = faiss.IndexFlatL2(vectors.shape[1])
    index.add(vectors)
    return index, words

def faiss_search(query_vector, index, words, k=10):
    distances, indices = index.search(np.array([query_vector]), k)
    top_k_words = [words[i] for i in indices[0]]
    return top_k_words, distances[0]

In [ ]:
index_es, words_es = create_faiss_index(esvec)
index_fr, words_fr = create_faiss_index(frvec)

In [ ]:
spanish_neighbors_dict = {}
french_neighbors_dict = {}

start_time_faiss_spanish = time.time()
for link in tqdm(estest, total=1000, desc="Processing Spanish Links", unit="link"):
    english_word = link[0]
    spanish_word = link[2]

    if english_word in envec:
        english_vector = envec[english_word]
        top_10_neighbors_spanish, _ = faiss_search(english_vector, index_es, words_es, k=10)
        spanish_neighbors_dict[english_word] = top_10_neighbors_spanish
end_time_faiss_spanish = time.time()

start_time_faiss_french = time.time()
for link in tqdm(frtest, total=1000, desc="Processing French Links", unit="link"):
    english_word = link[0]
    french_word = link[2]
    if english_word in envec:
        english_vector = envec[english_word]
        top_10_neighbors_french, _ = faiss_search(english_vector, index_fr, words_fr, k=10)
        french_neighbors_dict[english_word] = top_10_neighbors_french
end_time_faiss_french = time.time()

faiss_spanish_time = end_time_faiss_spanish - start_time_faiss_spanish
faiss_french_time = end_time_faiss_french - start_time_faiss_french

print("\n")
print(f"\nTime taken to compute top 10 neighbors for Spanish FAISS: {faiss_spanish_time:.4f} seconds")
print(f"Time taken to compute top 10 neighbors for French FAISS: {faiss_french_time:.4f} seconds")

Processing French Links: 100%|██████████| 1000/1000 [00:04<00:00, 219.31link/s]




Time taken to compute top 10 neighbors for Spanish FAISS: 4.2968 seconds
Time taken to compute top 10 neighbors for French FAISS: 4.5633 seconds


In [ ]:
def calculate_mrr_recall(test_set, neighbors_dict, valid_links, embeddings_dict, language):
    recall_at_10 = 0
    mrr_at_10 = 0
    valid_count = 0

    for link in tqdm(test_set, total=valid_links, desc=f"Calculating MRR & Recall for {language}", unit="link"):
        english_word = link[0]
        target_word = link[2]

        if english_word in embeddings_dict:
            valid_count += 1
            if english_word in neighbors_dict:
                top_10_neighbors = neighbors_dict[english_word]
                if target_word in top_10_neighbors:
                    recall_at_10 += 1

                for rank, word in enumerate(top_10_neighbors, start=1):
                    if word == target_word:
                        mrr_at_10 += 1 / rank
                        break

    recall_percentage = recall_at_10 / valid_count if valid_count > 0 else 0
    mrr_average = mrr_at_10 / valid_count if valid_count > 0 else 0
    return recall_percentage, mrr_average


recall_at_10_spanish_percentage, mrr_at_10_spanish_average = calculate_mrr_recall(estest, spanish_neighbors_dict, len(estest), envec, "Spanish")
recall_at_10_french_percentage, mrr_at_10_french_average = calculate_mrr_recall(frtest, french_neighbors_dict, len(frtest), envec, "French")

print("\n")
print(f"\nRecall@10 for Spanish: {recall_at_10_spanish_percentage:.2f}")
print(f"MRR@10 for Spanish: {mrr_at_10_spanish_average:.2f}")
print(f"\nRecall@10 for French: {recall_at_10_french_percentage:.2f}")
print(f"MRR@10 for French: {mrr_at_10_french_average:.2f}")

Calculating MRR & Recall for French: 100%|██████████| 1000/1000 [00:00<00:00, 380470.25link/s]




Recall@10 for Spanish: 0.61
MRR@10 for Spanish: 0.56

Recall@10 for French: 0.64
MRR@10 for French: 0.60


In [ ]:
print(f"Time Difference in FAISS as compared to Brute Force (Spanish): {time_taken_spanish - faiss_spanish_time:.2f} seconds")
print(f"Time Difference in FAISS as compared to Brute Force (French):  {time_taken_french - faiss_french_time:.2f} seconds")

Time Difference in FAISS as compared to Brute Force (Spanish): 232.65 seconds
Time Difference in FAISS as compared to Brute Force (French):  230.45 seconds


## Effectiveness for MRR@10 and Recall@10 for Brute force as well as FAISS:
Recall@10 is similar for both Brute force as well as FAISS. MRR@10 is also same for faiss as compared to brute force


## Time Complexity for Brute Force as well as FAISS:
The comparison between brute-force search and FAISS shows a remarkable difference in efficiency. This significant improvement is due to FAISS's optimization for fast approximate nearest neighbor search, using efficient indexing techniques that drastically cut down the time complexity. In contrast, brute force has to compare each query to every vector in the dataset, resulting in much longer computation times, especially with larger datasets. The time taken to get through FAISS is nearly 60 times faster than brute force, which is approximaterly 98% faster than brute force.

